In [1]:
import transformers
from transformers import pipeline
import os

pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

Device set to use cpu


In [2]:
total_score = 0
num_articles = 0

In [3]:
import pandas as pd

pwd = os.getcwd()

file_path = f'{pwd}/df.csv'
df = pd.read_csv(fr'{file_path}')

df.head()

,headline,date,time,description
0,Friedrich Merz’s cabinet: Who will lead German...,"April 11, 2025",4:40 am CET,Here are the politicians most likely to occupy...
1,Italy approves draft law outlawing violence ag...,"March 8, 2025",4:38 pm CET,NaN
2,Germany’s economy is driving off a cliff. The ...,"February 19, 2025",4:16 am CET,NaN
3,Trump’s return is an ‘electroshock’ for Europe...,"February 14, 2025",10:46 am CET,NaN
4,"Stocks tumble, dollar surges on Trump trade war","February 3, 2025",8:51 am CET,NaN


In [4]:
sentiment_label = []

for index, row in df.iterrows():
    print(f'Headline: {row["headline"]}')
    print(f'Date: {row["date"]}')
    print(f'Description: {row["description"]}')
    
    sentiment = pipe(row['headline'])[0]
    print(f'Sentiment: {sentiment}')
    
    print('-' * 40)
    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
        num_articles += 1
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']
        num_articles += 1
        
    sentiment_label.append(sentiment['label'])
    df.at[index, 'sentiment'] = sentiment['label']
    df.at[index, 'sentiment_score'] = sentiment['score']
    
df.to_csv(fr'{file_path}', index=False)

final_score = total_score / num_articles if num_articles > 0 else 0


Headline: Friedrich Merz’s cabinet: Who will lead Germany’s key ministries?
Date: April 11, 2025
Description: Here are the politicians most likely to occupy top posts in the next government.
Sentiment: {'label': 'neutral', 'score': 0.9408625364303589}
----------------------------------------
Headline: Italy approves draft law outlawing violence against women
Date: March 8, 2025
Description: nan
Sentiment: {'label': 'positive', 'score': 0.47708791494369507}
----------------------------------------
Headline: Germany’s economy is driving off a cliff. The car industry is behind the wheel.
Date: February 19, 2025
Description: nan
Sentiment: {'label': 'negative', 'score': 0.711748480796814}
----------------------------------------
Headline: Trump’s return is an ‘electroshock’ for Europe, says Macron
Date: February 14, 2025
Description: nan
Sentiment: {'label': 'neutral', 'score': 0.5189045667648315}
----------------------------------------
Headline: Stocks tumble, dollar surges on Trump trad

In [5]:
# After your existing code, add:

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
# Extract just the date part (removing time)
df['date'] = df['date'].dt.date

# Group by date and get majority sentiment
daily_sentiment = df.groupby('date')['sentiment'].agg(lambda x: x.value_counts().index[0]).reset_index()
# Get sentiment counts per day
daily_sentiment_counts = df.groupby(['date', 'sentiment']).size().unstack(fill_value=0)

# Print results
for date in daily_sentiment['date']:
    print(f'\nDate: {date}')
    print('Sentiment counts:')
    print(daily_sentiment_counts.loc[date])
    print(f"Majority sentiment: {daily_sentiment[daily_sentiment['date'] == date]['sentiment'].values[0]}")


Date: 2007-10-03
Sentiment counts:
sentiment
negative    0
neutral     1
positive    0
Name: 2007-10-03, dtype: int64
Majority sentiment: neutral

Date: 2007-12-05
Sentiment counts:
sentiment
negative    0
neutral     1
positive    0
Name: 2007-12-05, dtype: int64
Majority sentiment: neutral

Date: 2008-01-16
Sentiment counts:
sentiment
negative    0
neutral     2
positive    0
Name: 2008-01-16, dtype: int64
Majority sentiment: neutral

Date: 2008-02-13
Sentiment counts:
sentiment
negative    0
neutral     1
positive    0
Name: 2008-02-13, dtype: int64
Majority sentiment: neutral

Date: 2008-02-27
Sentiment counts:
sentiment
negative    0
neutral     1
positive    0
Name: 2008-02-27, dtype: int64
Majority sentiment: neutral

Date: 2008-03-05
Sentiment counts:
sentiment
negative    1
neutral     1
positive    0
Name: 2008-03-05, dtype: int64
Majority sentiment: negative

Date: 2008-03-18
Sentiment counts:
sentiment
negative    2
neutral     1
positive    1
Name: 2008-03-18, dtype: int6

In [6]:
# Group by date and get majority sentiment
majority_sentiment = df.groupby('date')['sentiment'].agg(lambda x: x.value_counts().index[0]).reset_index()
majority_sentiment.columns = ['date', 'majority_sentiment']

# Calculate average sentiment for each date
average_sentiment = df.groupby('date')['sentiment_score'].mean().reset_index()
average_sentiment.columns = ['date', 'average_sentiment']

# Merge the results
aggregated_sentiment = pd.merge(majority_sentiment, average_sentiment, on='date')

# Display the aggregated sentiment
print(aggregated_sentiment.head())

         date majority_sentiment  average_sentiment
0  2007-10-03            neutral           0.782351
1  2007-12-05            neutral           0.669374
2  2008-01-16            neutral           0.873376
3  2008-02-13            neutral           0.750085
4  2008-02-27            neutral           0.814265


In [7]:
# aggregate_sentiment to csv
aggregated_sentiment.to_csv(fr'{pwd}/aggregated_sentiment.csv', index=False)

In [8]:
aggregated_sentiment = pd.read_csv(fr'{pwd}/aggregated_sentiment.csv')
aggregated_sentiment.head()

,date,majority_sentiment,average_sentiment
0,2007-10-03,neutral,0.782351
1,2007-12-05,neutral,0.669374
2,2008-01-16,neutral,0.873376
3,2008-02-13,neutral,0.750085
4,2008-02-27,neutral,0.814265


In [9]:
stock_df = pd.read_csv(fr'{pwd}/stock_df.csv')
stock_df.head()

,date,Open,Close,Change
0,2025-03-03,3915.44,3917.741,0.058793
1,2025-02-28,3913.54,3915.439,0.048601
2,2025-02-27,3908.73,3913.537,0.122904
3,2025-02-26,3910.01,3908.733,-0.032736
4,2025-02-25,3920.37,3910.013,-0.264057
